In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import re
import regex
from pprint import pprint

import sys
src_path = str(Path.cwd().parent / "src")
sys.path.append(src_path)

import csv 

# Load data

Read the csv file (first row contains the column names), specify the data types.

In [2]:
csv_dir = Path.cwd().parent / "speeches_csv" 
speeches_path = csv_dir / "all_speeches.txt"
dtypes={'title':'string', 'pages':'int64', 'date':'string', 'location':'string', 
        'highest_speaker_count':'int64', 'content':'string'}
df = pd.read_csv(speeches_path, header=0, dtype=dtypes)
df.head()

,title,pages,date,location,highest_speaker_count,content
0,CGI_2013,19,24 September 2013,"Sheraton New York Hotel and Towers, New York, ...",10,Hillary Clinton: Thank you very much. I have t...
1,Prayer_Breakfast_2016,7,4 February 2016,"Washington Hilton, Washington, D.C.",0,"Well, good morning. Giving all praise and hono..."
2,Security_Team_Announcement,5,1 December 2008,"Chicago, Illinois",0,"Good morning, everybody. I hope you all had a ..."
3,Cairo_University,14,4 June 2009,"Cairo, Egypt",0,Thank you so much. Good afternoon. I am honore...
4,Umpqua_Community_College_Shootings,4,1 October 2015,"Washington, D.C.",0,There's been another mass shooting in America ...


In [3]:
df.dtypes

title                    string
pages                     int64
date                     string
location                 string
highest_speaker_count     int64
content                  string
dtype: object

# Dates

Some dates had the year missing.

In [4]:
temp = df.loc[:, ['title','date']]
temp['has_year'] = temp.apply(lambda row: row['date'][-4:].isnumeric(), axis=1)
temp.loc[temp.has_year==False, :]

,title,date,has_year
256,Recovery_and_Reinvestment_Act_2016,26 February,False
265,Post_Iran_Nuclear_Accord_Presser,15 July,False


Edit the dates that need to be corrected.

In [5]:
print(df.loc[df.title=='Community_College_Plan','date'])
df.loc[df.title=='Community_College_Plan','date'] = '9 January 2015'
print(df.loc[df.title=='Community_College_Plan','date'], '\n')

print(df.loc[df.title=='Recovery_and_Reinvestment_Act_2016','date'])
df.loc[df.title=='Recovery_and_Reinvestment_Act_2016','date'] = '26 February 2016'
print(df.loc[df.title=='Recovery_and_Reinvestment_Act_2016','date'], '\n')

print(df.loc[df.title=='Post_Iran_Nuclear_Accord_Presser','date'])
df.loc[df.title=='Post_Iran_Nuclear_Accord_Presser','date'] = '15 July 2015'
print(df.loc[df.title=='Post_Iran_Nuclear_Accord_Presser','date'], '\n')

78    9 January 20105
Name: date, dtype: string
78    9 January 2015
Name: date, dtype: string 

256    26 February 
Name: date, dtype: string
256    26 February 2016
Name: date, dtype: string 

265    15 July 
Name: date, dtype: string
265    15 July 2015
Name: date, dtype: string 



Parse the dates.

In [6]:
df['date'] = pd.to_datetime(df['date'], dayfirst=True, format="%d %B %Y")
df.head()

,title,pages,date,location,highest_speaker_count,content
0,CGI_2013,19,2013-09-24,"Sheraton New York Hotel and Towers, New York, ...",10,Hillary Clinton: Thank you very much. I have t...
1,Prayer_Breakfast_2016,7,2016-02-04,"Washington Hilton, Washington, D.C.",0,"Well, good morning. Giving all praise and hono..."
2,Security_Team_Announcement,5,2008-12-01,"Chicago, Illinois",0,"Good morning, everybody. I hope you all had a ..."
3,Cairo_University,14,2009-06-04,"Cairo, Egypt",0,Thank you so much. Good afternoon. I am honore...
4,Umpqua_Community_College_Shootings,4,2015-10-01,"Washington, D.C.",0,There's been another mass shooting in America ...


The `date` column now has type `datetime`.

In [7]:
df.dtypes

title                            string
pages                             int64
date                     datetime64[ns]
location                         string
highest_speaker_count             int64
content                          string
dtype: object

# Locations

Locations that specify a specific place in the White House can be replaced by `White House, Washington D.C.`.

In [8]:
contains_WH = df.location.str.contains("White House", flags=re.I)
df.loc[contains_WH, 'location'] = "White House, Washington D.C."

Make a `country`column, values for `White House` can already be filled.

In [9]:
df.loc[contains_WH, 'country'] = "USA"
df.loc[~contains_WH, 'country'] = ""

Set country to `USA` for locations that contain state names or state abbreviations. In case it contains the abbreviation, replace it by the full state name.

In [10]:
states_full = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']
states_abbr = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

for state in states_full:
    contains = df.location.str.contains(state, flags=re.I)
    df.loc[contains, 'country'] = "USA"
    
for i in range(len(states_abbr)):
    contains = df.location.str.contains(r", \b"+states_abbr[i]+r"\b", flags=re.I)
    df.loc[contains, 'country'] = "USA"
    df['location'] = df.location.str.replace(r", \b"+states_abbr[i]+r"\b", repl=", "+states_full[i], flags=re.I, regex=True)

In [11]:
df.loc[df.country=="USA", :]

,title,pages,date,location,highest_speaker_count,content,country
0,CGI_2013,19,2013-09-24,"Sheraton New York Hotel and Towers, New York, ...",10,Hillary Clinton: Thank you very much. I have t...,USA
1,Prayer_Breakfast_2016,7,2016-02-04,"Washington Hilton, Washington, D.C.",0,"Well, good morning. Giving all praise and hono...",USA
2,Security_Team_Announcement,5,2008-12-01,"Chicago, Illinois",0,"Good morning, everybody. I hope you all had a ...",USA
4,Umpqua_Community_College_Shootings,4,2015-10-01,"Washington, D.C.",0,There's been another mass shooting in America ...,USA
5,White_House_Correspondent_Dinner_2013,6,2013-04-27,"Washington Hilton Hotel, Washington, D.C.",0,"Thank you. Thank you, everybody. How do you li...",USA
...,...,...,...,...,...,...,...
421,VFW_2012,9,2012-07-23,"Reno, Nevada",0,Thank you! Hello VFW! Thank you so much. Pleas...,USA
423,Wakefield_Back_to_High_School,7,2009-09-08,"Arlington, Virginia",0,"Hello, everybody! Thank you. Thank you. Thank ...",USA
424,CIA_First_Speech,5,2009-04-20,"Langley, Virginia",1,Thank you for the extraordinary welcome. And t...,USA
429,Second_Democratic_Nomination_Acceptance,9,2012-09-06,"Charlotte, North Carolina",0,"Thank you, so much. Thank you. Thank you very ...",USA


In [12]:
df.loc[(df.country!="USA") & (df.location!="unknown_location"), :]

,title,pages,date,location,highest_speaker_count,content,country
3,Cairo_University,14,2009-06-04,"Cairo, Egypt",0,Thank you so much. Good afternoon. I am honore...,
14,PM_Abe_of_Japan_Joint_Presser,14,2014-04-24,"Akasaka Palace, Tokyo, Japan",10,Prime Minister Abe: As interpreted. On behalf ...,
15,UN_Copenhagen_Climate_Change,3,2009-12-18,"Copenhagen, Denmark",0,Good morning. It is an honor for me to join th...,
16,People_of_Greece,15,2016-11-16,"Stavros Niarchos Foundation Cultural Center, A...",0,"Hello, Greece! Yia sas! Kalispera! Good evenin...",
17,India_Speech_at_New_Delhi,9,2015-01-27,"Siri Fort Auditorium, New Delhi, India",0,"Thank you so much. Thank you so much, Neha, fo...",
...,...,...,...,...,...,...,...
409,Saudi_Arabia_Presser,6,2005-04-21,"Diriyah Palace, Riyadh, Saudi Arabia",5,President Obama: You already heard my statemen...,
426,Hiroshima_Peace_Memorial_Address,5,2016-05-27,"Hiroshima, Japan",0,"Seventy-one years ago, on a bright, cloudless ...",
430,Shimon_Peres_Memorial,6,2016-09-30,"Mount Herzl, Jerusalem, Israel",0,"Zvia, Yoni, Chemi and generations of the Peres...",
431,ASEAN_Business_2015,9,2015-11-21,"Shangri-La Hotel, Kuala Lumpur, Malaysia",0,"Thank you so much. Please be seated. Well, goo...",


Change `Washington, D.C.` to `Washington D.C.`.

In [13]:
df['location'] = df.location.str.replace("Washington, D.C.|Washington, DC", repl="Washington D.C.", flags=re.I, regex=True)

If `country=='USA'`: We assume the last substring to be the state, the second to last the city and everything before that a more specific locations.


If `country!='USA'`: We assume the last substring to be the country, the second to last the city and everything before that a more specific locations.

In [14]:
df.loc[:, 'count_commas'] = df.loc[:, 'location'].str.count(',')
df.loc[:,['location','country','count_commas']].sort_values(by='count_commas')

,location,country,count_commas
217,unknown_location,,0
179,unknown_location,,0
181,Washington D.C.,USA,0
189,unknown_location,,0
198,unknown_location,,0
...,...,...,...
56,"McCormick Place, Chicago, Illinois",USA,2
246,"Phillips Center for the Performing Arts, Orlan...",USA,2
316,"Gostinny Dvor, Moscow, Russia",,2
0,"Sheraton New York Hotel and Towers, New York, ...",USA,2


## USA

No commas.

In [15]:
print(df.loc[(df.country=="USA") & (df.count_commas == 0), ['title','location']].sort_values(by='location'), '\n')

df.loc[df.title=='Ebola_CDC', ['state','city','specific_location']] = ['Georgia', 'Atlanta', 'no_specific_location']
df.loc[df.location=='Washington D.C.', ['state','city','specific_location']] = ['no_state', 'Washington D.C.', 
                                                                                'no_specific_location']

                                      title         location
308                               Ebola_CDC  Atlanta Georgia
258                 State_of_the_Union_2012  Washington D.C.
278                 Health_Care_Law_Signing  Washington D.C.
284  White_House_Correspondents_Dinner_2015  Washington D.C.
285                 Paris_Terrorist_Attacks  Washington D.C.
288                     Brookings_Institute  Washington D.C.
293   Go_Presidential_Election_Outcome_2016  Washington D.C.
298          Howard_University_Commencement  Washington D.C.
4        Umpqua_Community_College_Shootings  Washington D.C.
314      National_Holocaust_Memorial_Museum  Washington D.C.
321     Iftar_Dinner_on_Religious_Tolerance  Washington D.C.
322   Second_Presidential_Inaugural_Address  Washington D.C.
330   White_House_Correspondent_Dinner_2014  Washington D.C.
356              Syria_Speech_to_the_Nation  Washington D.C.
370        Final_State_of_the_Union_Address  Washington D.C.
371                     

One comma + `Washington D.C.`

In [16]:
contains_WDC = df.location.str.contains("Washington D.C.", flags=re.I)
select = contains_WDC & (df.count_commas == 1)
print(df.loc[select, 'location'])

locations = df.loc[select, 'location'].str.extract(r"(.+), Washington D.C. *", flags=re.I)

df.loc[select, ['state','city']]  = ['no_state', 'Washington D.C.']
df.loc[select, 'specific_location'] = locations.values

1               Washington Hilton, Washington D.C.
5         Washington Hilton Hotel, Washington D.C.
8                     White House, Washington D.C.
11                    White House, Washington D.C.
12                    White House, Washington D.C.
                          ...                     
414                   White House, Washington D.C.
415              State Department, Washington D.C.
416           Eisenhower Building, Washington D.C.
418    U.S. Capitol Western Front, Washington D.C.
434                   White House, Washington D.C.
Name: location, Length: 106, dtype: string 



One comma + other

In [17]:
select = (df.country=="USA") & ~contains_WDC & (df.count_commas == 1)
print(df.loc[select, 'location'])

states = df.loc[select, 'location'].str.extract(r".+?, *(.+)", flags=re.I)
cities = df.loc[select, 'location'].str.extract(r"(.+?), *.+", flags=re.I)

df.loc[select, 'state']  = states.values
df.loc[select, 'city']  = cities.values
df.loc[select, 'specific_location'] = 'no_specific_location'

2              Chicago, Illinois
10          San Jose, California
20             Fairfax, Virginia
28             Beaverton, Oregon
31                Kailua, Hawaii
                 ...            
420          Arlington, Virginia
421                 Reno, Nevada
423          Arlington, Virginia
424            Langley, Virginia
429    Charlotte, North Carolina
Name: location, Length: 74, dtype: string


In [18]:
df.loc[select, ['location','country','state','city','specific_location']].sort_values(
    by= ['state','city','specific_location'])

,location,country,state,city,specific_location
61,"Alberta, Alabama",USA,Alabama,Alberta,no_specific_location
227,"Hilton San Francisco Union Square, California",USA,California,Hilton San Francisco Union Square,no_specific_location
163,"Rancho Mirage, California",USA,California,Rancho Mirage,no_specific_location
224,"Rancho Mirage, California",USA,California,Rancho Mirage,no_specific_location
10,"San Jose, California",USA,California,San Jose,no_specific_location
...,...,...,...,...,...
153,"Joint Base Myer-Henderson Hall, Virginia",USA,Virginia,Joint Base Myer-Henderson Hall,no_specific_location
242,"Langley, Virginia",USA,Virginia,Langley,no_specific_location
424,"Langley, Virginia",USA,Virginia,Langley,no_specific_location
204,"McLean, Virginia",USA,Virginia,McLean,no_specific_location


Some cities need corrections.

Two commas

In [19]:
select = (df.country=="USA") & (df.count_commas == 2)
print(df.loc[select, 'location'])

states = df.loc[select, 'location'].str.extract(r".+?,.+?, *(.+)", flags=re.I)
cities = df.loc[select, 'location'].str.extract(r".+?, *(.+?),.+", flags=re.I)
specific_locs = df.loc[select, 'location'].str.extract(r" *(.+?),.+?,.+", flags=re.I)

df.loc[select, 'state']  = states.values
df.loc[select, 'city']  = cities.values
df.loc[select, 'specific_location'] = specific_locs.values

0      Sheraton New York Hotel and Towers, New York, ...
24          Harborside Event Center, Fort Myers, Florida
26              Kaneohe Bay Marine Base, Kaneohe, Hawaii
38        Veterans Memorial Auditorium, Des Moines, Iowa
53     Newport News Shipbuilding, Newport News, Virginia
56                    McCormick Place, Chicago, Illinois
65               Hofstra University, Hempstead, New York
71                         Del Sol HS, Las Vegas, Nevada
78     Pellissippi State Community College, Knoxville...
80          Lyndon Baines Johnson Library, Austin, Texas
92                    Lincoln Center, New York, New York
99                         Pentagon, Arlington, Virginia
102       Joint Base Myer-Henderson, Fort Myer, Virginia
106    Philadelphia Convention Center, Philadelphia, ...
107                    Sun Devil Stadium, Tempe, Arizona
109        Illinois State Capitol, Springfield, Illinois
111          Hyde Park Career Academy, Chicago, Illinois
137     Arlington National Ceme

In [20]:
df.loc[select, ['location','country','state','city','specific_location']].sort_values(
    by= ['state','city','specific_location'])

,location,country,state,city,specific_location
367,"Edmund Pettus Bridge, Selma, Alabama",USA,Alabama,Selma,Edmund Pettus Bridge
107,"Sun Devil Stadium, Tempe, Arizona",USA,Arizona,Tempe,Sun Devil Stadium
381,"Stanford University, Stanford, California",USA,California,Stanford,Stanford University
220,"United States Air Force Academy, Colorado Spri...",USA,Colorado,Colorado Springs,United States Air Force Academy
237,"Magness Arena, University of Denver, Colorado",USA,Colorado,University of Denver,Magness Arena
190,"Lynn University, Boca Raton, Florida",USA,Florida,Boca Raton,Lynn University
24,"Harborside Event Center, Fort Myers, Florida",USA,Florida,Fort Myers,Harborside Event Center
245,"JFK Space Center, Merritt Island, Florida",USA,Florida,Merritt Island,JFK Space Center
246,"Phillips Center for the Performing Arts, Orlan...",USA,Florida,Orlando,Phillips Center for the Performing Arts
259,"MacDill Air Force Base, Tampa, Florida",USA,Florida,Tampa,MacDill Air Force Base


Some cities need corrections.

In [21]:
df.loc[df.country=="USA", ['location','country','state','city','specific_location']].sort_values(
    by=['state','city','specific_location'])

,location,country,state,city,specific_location
61,"Alberta, Alabama",USA,Alabama,Alberta,no_specific_location
367,"Edmund Pettus Bridge, Selma, Alabama",USA,Alabama,Selma,Edmund Pettus Bridge
107,"Sun Devil Stadium, Tempe, Arizona",USA,Arizona,Tempe,Sun Devil Stadium
227,"Hilton San Francisco Union Square, California",USA,California,Hilton San Francisco Union Square,no_specific_location
163,"Rancho Mirage, California",USA,California,Rancho Mirage,no_specific_location
...,...,...,...,...,...
356,Washington D.C.,USA,no_state,Washington D.C.,no_specific_location
370,Washington D.C.,USA,no_state,Washington D.C.,no_specific_location
371,Washington D.C.,USA,no_state,Washington D.C.,no_specific_location
401,Washington D.C.,USA,no_state,Washington D.C.,no_specific_location


## Not USA, but known location

Some US locations don't have `country=='USA'`

In [27]:
select = (df.country!="USA") & (df.location!="unknown_location") & (df.count_commas == 0)
df.loc[select, ['location','country','state','city','specific_location']]

,location,country,state,city,specific_location
44,Manila,,NaN,NaN,NaN
114,James S. Brady Press Briefing Room,,NaN,NaN,NaN
168,Victory Column,,NaN,NaN,NaN
199,James L. Knight International Center,,NaN,NaN,NaN
200,Rhenus Sports Arena,,NaN,NaN,NaN
273,Czech Republic,,NaN,NaN,NaN
348,James S. Brady Press Briefing Room,,NaN,NaN,NaN


## Unknown locations

In [22]:
print('There are %i unknown locations.' % len(df.loc[df.location=="unknown_location", :]))

There are 89 unknown locations.
